In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import sympy as sp
from sympy.vector import CoordSys3D

from matplotlib import animation, rc
from IPython.display import HTML, display
rc('animation', html='jshtml')
import seaborn as sns
C = CoordSys3D('')

#Define plotting style:
sns.set() #Set style
sns.set_style('ticks',{'font.family':'serif', 'font.serif':'Times New Roman'})
sns.set_context('poster', font_scale=0.8,  rc={"lines.linewidth": 5})

## Problema (Mola atrito)

<font size=3px>Uma mola de constante elástica $k = 20$ N/m está presa a um bloco de massa $m = 5$ kg. O bloco é
    colocado na posição $x_0 = -1$ m e solto a partir do repouso. A mola encontra-se relaxada quando a posição do bloco
    é $x=0$. Se a superfície possui um coeficiente de atrito cinético $\mu_k = 0.02$, calcule:<br>
    a) O trabalho realizado pela força de atrito entre a $x_0$ e a elongação máxima da mola ($x_{max}$). Escreva a resposta em função de $x_{max}$.<br>
    b) O trabalho realizado pela força elástica entre a $x_0$ e a elongação máxima da mola ($x_{max}$).  Escreva a resposta em função de $x_{max}$.<br>
    c) Determine o valor de $x_{max}$.<br>
    d) Determine para qual valor de $x$ a energia cinética do bloco será máxima.
</font>

In [2]:
#Definir variaveis/parametros:
m,k,x0,v,x,xm,muk,g = sp.symbols('m,k,x_0,v,x,x_{max},mu_k,g')
subsList = [(m,5.0),(x0,-1.),(k,20.),(muk,0.02),(g,9.8)]

In [3]:
#item a)
#Entre a posição inicial e xmax a força de atrito sempre é oposta ao eixo x e constante:
N = m*g
Fatv = -N*muk*C.i
#Então o trabalho da força será:
Wf = Fatv.dot((xm-x0)*C.i)
resp = display(Wf)
Wf.subs(subsList)

-g*m*mu_k*(-x_0 + x_{max})

-0.98*x_{max} - 0.98

In [4]:
#item b)
#A força elástica é dada por:
Fel = -k*x*C.i
#Então o trabalho da força será:
We = sp.integrate(Fel.dot(C.i),(x,x0,xm))
resp = display(We)
We.subs(subsList)

k*x_0**2/2 - k*x_{max}**2/2

10.0 - 10.0*x_{max}**2

In [5]:
#item c)
#xmax será dado pelo ponto onde a energia cinética é nula, ou seja deltaK = 0 (pois K0=0):
Wtotal = We + Wf
dK = 0
xmsol = sp.solve(sp.Eq(Wtotal,dK),xm)[1]
resp = display(xmsol)
xmsol.subs(subsList)

-2*g*m*mu_k/k - x_0

0.902000000000000

In [6]:
#item d)
#A energia cinética quando o bloco está na posição x é:
K = Wtotal.subs(xm,x)
resp = display(K)
xsol = sp.solve(sp.Eq(sp.diff(K,x),0),x)[0]
resp = display(xsol)
xsol.subs(subsList)

-g*m*mu_k*(x - x_0) - k*x**2/2 + k*x_0**2/2

-g*m*mu_k/k

-0.0490000000000000

## Problema (Pêndulo)

<font size=3px>Um pêndulo feito de uma esfera de massa $m$ presa a um fio de massa 
    desprezível e comprimento $l$, é solto do ponto indicado na figura. <br>
    a) Escreva o diagrama de forças que atuam no pêndulo quando este se encontra a um ângulo $\theta$.<br>
    b) Calcule o trabalho realizado pela tração durante o ponto inicial e o de menor altura do pêndulo.<br>
    c) Calcule o trabalho realizado pela força peso durante o ponto inicial e o de menor altura do pêndulo.<br>
    d) Qual o vetor velocidade do pêndulo quando ele se encontra em seu ponto de menor altura?
</font>
<img src="pendulo.png" alt="Drawing" style="width: 250px;"/>

In [7]:
#Definir variaveis/parametros:
m,l,tt,v,T,g,dtt,tt0 = sp.symbols('m,l,theta,v,T,g,dθ,theta_0')

In [8]:
#item a)
#Adotando um sistema onde y aponta para cima (vertical):
Tv = T*sp.cos(tt)*C.j - T*sp.sin(tt)*C.i
Pv = -m*g*C.j
resp = display(Tv)
resp = display(Pv)

(-T*sin(theta))*.i + (T*cos(theta))*.j

(-g*m)*.j

In [9]:
#item b)
#O vetor deslocamento é dado por:
drv = -l*dtt*(-sp.cos(tt)*C.i+-sp.sin(tt)*C.j)
#O produto escalar da tensão com dr é:
Tv.dot(drv)
#Logo WT = 0

0

In [10]:
#item c)
#O produto escalar do peso com dr é:
resp = display(Pv.dot(drv))
#Então o trabalho do peso é:
WP = sp.integrate(Pv.dot(drv)/dtt,(tt,tt0,0))
resp = display(sp.simplify(WP))

-dθ*g*l*m*sin(theta)

g*l*m*(1 - cos(theta_0))

In [11]:
#item d)
#No ponto de menor altura:
K = WP
vsol = sp.solve(sp.Eq(m*v**2/2,K),v)[0]
#E como a velocidade aponta contra o eixo x:
v = vsol*(-C.i)
v

(-sqrt(2)*sqrt(g*l*(1 - cos(theta_0))))*.i

## Problema (Mola - queda livre)

<font size=3px> Um bloco de massa $m$ é deixado cair em uma mola vertical, inicialmente relaxada, com uma constante elástica $k$. O bloco fica aclopado à mola, comprimindo-a de $d$ até parar momentaneamente.<br>
    a) Especifique um sistema de coordenadas e determine os vetores força que atuam no bloco durante a compressão da mola.<br>
    b) Determine o trabalho total realizado pela força elástica até a compressão máxima da mola.<br>
    c) Qual é a velocidade do bloco imediatamente antes de se chocar com a mola?<br>
    d) Qual é a altura máxima do bloco após este ser lançado verticalmente para cima pela mola?
</font>
<img src="blocoMola.png" alt="Drawing" style="width: 100px;"/>

In [12]:
#Definir variaveis/parametros:
m,d,k,g,v0,x,h = sp.symbols('m,d,k,g,v_0,x,h')

In [13]:
#item a)
#Adotando um sistema de coordenadas x vertical que aponta para cima e com origem no ponto de relaxamento da mola:
Pv = -m*g*C.i
Fk = -k*x*C.i
resp = display(Pv)
resp = display(Fk)

(-g*m)*.i

(-k*x)*.i

In [14]:
#item b)
#Adotando o ponto de referência da energia potencial como a origem, temos que a variação da energia
#potencial elástica é:
Uk0 = 0 #energia inicial
Ukf = k*d**2/2 #energia no ponto de máxima compressão (x = -d)
#Logo o trabalho é:
Wk = -(Ukf-Uk0)
resp = display(Wk)

-d**2*k/2

In [15]:
#item c)
#Como todas as forças são conservativas, a variação de energia mecânica é zero.
#No ponto inicial:
U0 = 0
K0 = m*v0**2/2
#No ponto de máxima compressão (x = -d):
Uf = -m*g*d +k*d**2/2
Kf = 0
#Então:
vsol = sp.solve(sp.Eq(U0+K0,Uf+Kf),v0)[1]
vsol

sqrt(d*(d*k - 2*g*m)/m)

In [16]:
#A energia total do bloco é:
E0 = K0.subs(v0,vsol) + U0
#No ponto de máxima altura:
Kf = 0
Uf = m*g*h
hsol = sp.solve(sp.Eq(E0,Uf+Kf),h)[0]
hsol

d**2*k/(2*g*m) - d

## Problema (Duas molas)

<font size=3px> Considere um bloco sobre uma mesa. Esse bloco sai do repouso e é empurrado por uma mola presa à parede,
desliza pela mesa e depois cai verticalmente no chão, onde outra mola, de comprimento $L_0 = 0,3$ m, aguarda sua chegada. O bloco tem massa $m = 1,35$ kg. A constante elástica da mola da mesa é $k_1 = 560$ N/m. O bloco desliza uma distância $d = 0,65$ m pela mesa desde o instante inicial. A altura da mesa é $h = 0,75$ m e o coeficiente de atrito cinético entre o bloco e a mesa é $\mu_k = 0,16$.<br>
    a) Calcule o trabalho realizado pela força de atrito.<br>
    b) Calcule qual foi a compressão inicial da mola na mesa.<br>
    c) Calcule a compressão máxima da mola localizada no chão se a sua constante elástica é $k_2 = 600$ N/m.
</font>
<img src="duasMolas.png" alt="Drawing" style="width: 200px;"/>

In [17]:
#Definir variaveis/parametros:
m,L0,k1,d,k2,g,v0,muk,h,d0,y = sp.symbols('m,L_0,k_1,d,k_2,g,v_0,mu_k,h,d_0,y')
subsList = [(L0,0.3),(m,1.35),(k1,560),(d,0.65),(h,0.75),(muk,0.16),(g,9.8),(k2,600)]

In [18]:
#item a)
#Adotando um sistema de coordenadas com origem no ponto de relaxamento da mola da mesa e no chão, com o eixo x apontando para a direita
#e o eixo y para cima, temos que o trabalho da força de atrito será:
Fat = -m*g*muk*C.i
dx = d*C.i
Wfat = Fat.dot(dx)
resp = display(Wfat)
Wfat.subs(subsList).evalf(2)

-d*g*m*mu_k

-1.4

In [19]:
#item b)
#Para que o bloco caia na segunda mola, sua velocidade ao final da mesa deve ser nula.
#No instante inicial temos:
U0 = k1*d0**2/2 + m*g*h
K0 = 0
#Já no instante em que chega ao final da mesa:
Uf = m*g*h
Kf = 0
#A variação de energia total deve ser igual ao trabalho da força de atrito:
dsol = sp.solve(sp.Eq(Uf+Kf-(U0+K0),Wfat),d0)[1]
resp = display(dsol)
dsol.subs(subsList).evalf(2)

sqrt(2)*sqrt(d*g*m*mu_k/k_1)

0.070

In [20]:
#item c)
#A energia potencial da segunda mola é dada por:
Uk = k2*((y-L0)**2)/2
#Já a energia potencial gravitacional é:
Ug = m*g*y
#No ponto de máxima compressão da mola, temos:
E2 = Uk+Ug
#e como a energia inicial (após sair da mesa) é conservada:
Ef = m*g*h
#temos:
ysol = sp.solve(sp.Eq(Ef,E2),y)[0]
resp = display(ysol)
ysol.subs(subsList).evalf(2)

(L_0*k_2 - g*m - sqrt(g*m*(-2*L_0*k_2 + g*m + 2*h*k_2)))/k_2

0.14

In [21]:
#Logo a compressão máxima da mola será:
dmax = sp.simplify(L0-ysol)
resp = display(dmax)
dmax.subs(subsList).evalf(2)

(g*m + sqrt(g*m*(-2*L_0*k_2 + g*m + 2*h*k_2)))/k_2

0.16